In [5]:
# imports
import time
import sklearn
import pandas as pd
import numpy
import multiprocess as mp

from PIL import Image, ImageOps, ImageEnhance
from sklearn.metrics import classification_report, accuracy_score
from skimage.feature import hog
from sklearn import svm
from matplotlib import cm


# directorios
train_csv_path = '../data/processed/train_split.txt'
test_csv_path = '../data/processed/test_split.txt'

train_hoged_nmf_csv_path = '../data/processed/nmf/train_split.txt'
test_hoged_nmf_csv_path = '../data/processed/nmf/test_split.txt'


In [6]:
# Obtengo la base en gris y resizeada, y por otro lado los features de hog pasado por nmf
train_df = pd.read_csv(train_csv_path, delimiter = ',', header='infer')
print('train_df listo')
test_df = pd.read_csv(test_csv_path, delimiter = ',', header='infer')
print('test_df listo')

train_feats_df = pd.read_csv(train_hoged_nmf_csv_path, delimiter = ',', header='infer')
print('train_feats_df listo')
test_feats_df = pd.read_csv(test_hoged_nmf_csv_path, delimiter = ',', header='infer')
print('test_feats_df listo')

train_df listo
test_df listo
train_feats_df listo
test_feats_df listo


In [7]:
# Agrego los features al dataframe con toda la info
train_df['features'] = train_feats_df['features']
test_df['features'] = test_feats_df['features']

In [8]:
# Reviso el contenido de una celda
train_df['features'][89]

'[0.06035080672897604, 0.01349485083242839, 0.0815040414422112, 0.2340637654417483, 0.2358676316795574, 0.21317269957438714, 0.06827914233247252, 0.01349485083242839, 0.0, 0.20849200349921662, 0.07573919882819971, 0.27573820023361556, 0.06507935404307867, 0.1327717748037473, 0.12283055041682023, 0.03413957116623626, 0.15488000260361792, 0.13190362647826726, 0.024140322691590416, 0.05397940332971356, 0.32990430618143457, 0.32990430618143457, 0.32990430618143457, 0.0, 0.008534892791559065, 0.0, 0.0, 0.036210484037385623, 0.12463935487486068, 0.32990430618143457, 0.32990430618143457, 0.21863407186422584, 0.10005370601522448, 0.03864337190479647, 0.01349485083242839, 0.0, 0.17642147621311025, 0.06408889089369543, 0.23332377029866957, 0.05506875812298826, 0.1123486374401872, 0.10393658588774464, 0.028888175284154553, 0.13105614717412073, 0.11161402888646538, 0.10724188119290597, 0.0368253426684128, 0.09775421092397357, 0.1264882571146377, 0.2093770061385307, 0.11696127473439621, 0.084126194

In [9]:
# Al estar como string, reviso como convertirlo a un array
numpy.array(train_df['features'][89].replace('[', '').replace(']', '').split(', ')).astype(float)

array([0.06035081, 0.01349485, 0.08150404, ..., 0.01104093, 0.05045093,
       0.44276709])

In [10]:
# Habiendo probado 1, paso toda la columna
train_df['features'] = train_df['features'].apply(lambda x: numpy.array(x.replace('[', '').replace(']', '').split(', ')).astype(float))
train_df.info(verbose=True)
train_df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13889 entries, 0 to 13888
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  13889 non-null  int64 
 1   patient_id  13889 non-null  object
 2   image_id    13889 non-null  object
 3   disease     13889 non-null  object
 4   method      13889 non-null  object
 5   ancho       13889 non-null  int64 
 6   alto        13889 non-null  int64 
 7   features    13889 non-null  object
dtypes: int64(3), object(5)
memory usage: 868.2+ KB


,Unnamed: 0,patient_id,image_id,disease,method,ancho,alto,features
0,0,3,SARS-10.1148rg.242035193-g04mr34g0-Fig8a-day0....,pneumonia,cohen,1692,1474,"[0.12418160035654287, 0.11057938516591702, 0.1..."
1,1,3,SARS-10.1148rg.242035193-g04mr34g0-Fig8b-day5....,pneumonia,cohen,1800,1546,"[0.08821158414441169, 0.09161997739854053, 0.1..."
2,2,3,SARS-10.1148rg.242035193-g04mr34g0-Fig8c-day10...,pneumonia,cohen,1800,1546,"[0.13655804837282656, 0.2375007180962649, 0.23..."
3,3,7,SARS-10.1148rg.242035193-g04mr34g04a-Fig4a-day...,pneumonia,cohen,1772,1800,"[0.3222769727620392, 0.022967671078387587, 0.0..."
4,4,7,SARS-10.1148rg.242035193-g04mr34g04b-Fig4b-day...,pneumonia,cohen,1708,1710,"[0.025275084346299537, 0.007594217648116013, 0..."
5,5,9,SARS-10.1148rg.242035193-g04mr34g07a-Fig7a-day...,pneumonia,cohen,1800,1508,"[0.14194839250419267, 0.031740625959642034, 0...."
6,6,9,SARS-10.1148rg.242035193-g04mr34g07b-Fig7b-day...,pneumonia,cohen,1800,1522,"[0.04421264967340874, 0.038071655496190904, 0...."
7,7,10,SARS-10.1148rg.242035193-g04mr34g09a-Fig9a-day...,pneumonia,cohen,1750,1800,"[0.0236001674646301, 0.031850688950337075, 0.0..."
8,8,10,SARS-10.1148rg.242035193-g04mr34g09b-Fig9b-day...,pneumonia,cohen,1800,1779,"[0.1675767277688176, 0.02661098442360383, 0.08..."
9,9,10,SARS-10.1148rg.242035193-g04mr34g09c-Fig9c-day...,pneumonia,cohen,1787,1800,"[0.08491244512093893, 0.15591200635484065, 0.2..."


In [11]:
test_df['features'] = test_df['features'].apply(lambda x: numpy.array(x.replace('[', '').replace(']', '').split(', ')).astype(float))
test_df.info(verbose=True)
test_df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1559 entries, 0 to 1558
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  1559 non-null   int64 
 1   patient_id  1559 non-null   object
 2   image_id    1559 non-null   object
 3   disease     1559 non-null   object
 4   method      1559 non-null   object
 5   ancho       1559 non-null   int64 
 6   alto        1559 non-null   int64 
 7   features    1559 non-null   object
dtypes: int64(3), object(5)
memory usage: 97.6+ KB


,Unnamed: 0,patient_id,image_id,disease,method,ancho,alto,features
0,0,8,SARS-10.1148rg.242035193-g04mr34g05x-Fig5-day9...,pneumonia,cohen,1800,1593,"[0.02705105981815912, 0.0, 0.0, 0.039409680643..."
1,1,31,streptococcus-pneumoniae-pneumonia-temporal-ev...,pneumonia,cohen,1534,1517,"[0.2606366602440652, 0.3124750936954903, 0.124..."
2,2,31,streptococcus-pneumoniae-pneumonia-temporal-ev...,pneumonia,cohen,1534,1517,"[0.45842221000070377, 0.13740779670673198, 0.2..."
3,3,31,streptococcus-pneumoniae-pneumonia-temporal-ev...,pneumonia,cohen,1534,1517,"[0.3849850164553064, 0.04914836231399041, 0.0,..."
4,4,31,streptococcus-pneumoniae-pneumonia-temporal-ev...,pneumonia,cohen,1534,1517,"[0.4414475478504372, 0.19924520498368148, 0.05..."
5,5,19,1-s2.0-S0929664620300449-gr2_lrg-a.jpg,COVID-19,cohen,1165,1163,"[0.11631075450277166, 0.20638893579784282, 0.2..."
6,6,19,1-s2.0-S0929664620300449-gr2_lrg-b.jpg,COVID-19,cohen,1170,1161,"[0.31774886959572407, 0.0034085860092542782, 0..."
7,7,19,1-s2.0-S0929664620300449-gr2_lrg-c.jpg,COVID-19,cohen,1165,1165,"[0.19894642529984977, 0.08813710110747786, 0.2..."
8,8,19,1-s2.0-S0929664620300449-gr2_lrg-d.jpg,COVID-19,cohen,1166,1161,"[0.06812645446495401, 0.006496334669921768, 0...."
9,9,20,covid-19-pneumonia-15-PA.jpg,COVID-19,cohen,2800,3368,"[0.07546193180267767, 0.015287980365369252, 0...."


In [12]:
# Verifico longitudes de diferentes filas
print(len(train_df['features'][22]))
print(len(train_df['features'][55]))
print(len(train_df['features'][677]))
print(len(train_df['features'][9000]))
print(len(train_df['features'][10002]))


print(len(test_df['features'][22]))
print(len(test_df['features'][55]))
print(len(test_df['features'][100]))


print(len(train_df['disease']))
print(len(train_df['features']))

86436
86436
86436
86436
86436
86436
86436
86436
13889
13889


In [13]:
# Reviso la columna de enfermedad
print(test_df['disease'].value_counts())

normal       885
pneumonia    594
COVID-19      80
Name: disease, dtype: int64


In [14]:
# convierto esos valores de str a un valor numérico
normal = 'normal'
pneumonia = 'pneumonia'
covid = 'COVID-19'

def disease_to_num(x):
    if x == normal:
        return 0
    elif x == pneumonia:
        return 1
    else:
        return 2

disease_to_num(train_df['disease'][0])

1

In [15]:
# Habiendo probado con 1, convierto toda la columna
train_df['disease'] = train_df['disease'].apply(lambda x: disease_to_num(x))

In [16]:
test_df['disease'] = test_df['disease'].apply(lambda x: disease_to_num(x))

In [17]:
# Verifico
print(train_df['disease'][9])
print(test_df['disease'][92])

1
0


In [18]:
# paso a numpy array
train_set = train_df['features'].to_numpy()
train_set[0]

array([0.1241816 , 0.11057939, 0.12777448, ..., 0.22682526, 0.06005816,
       0.03251163])

In [19]:
# Verifico tamaño
print(len(train_set))
len(train_set[0])

13889


86436

In [20]:
test_set = test_df['features'].to_numpy()
test_set[0]

array([0.02705106, 0.        , 0.        , ..., 0.        , 0.        ,
       0.        ])

In [21]:
# Ejecuto el modelo de random forest
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier

start = time.time()
clf = RandomForestClassifier(min_samples_leaf=20)
clf.fit(list(train_set), train_df['disease'])
end = time.time()

print("Random Forest", end - start, clf.score(list(train_set), train_df['disease']))
proba = clf.predict_proba(list(train_set))
print("Proba")
print(proba)

y_pred = clf.predict(list(test_set))

print("Accuracy: "+ str(accuracy_score(test_df['disease'], y_pred)))
print('\n')
print(classification_report(test_df['disease'], y_pred))

Random Forest 223.33643698692322 0.8806969544243646
Proba
[[0.31725274 0.50097394 0.18177332]
 [0.42065191 0.44151967 0.13782842]
 [0.31264064 0.5802853  0.10707405]
 ...
 [0.16484545 0.8152596  0.01989495]
 [0.4545394  0.46623668 0.07922393]
 [0.26606056 0.71377212 0.02016732]]
Accuracy: 0.7767799871712636


              precision    recall  f1-score   support

           0       0.77      0.88      0.82       885
           1       0.79      0.73      0.76       594
           2       0.00      0.00      0.00        80

    accuracy                           0.78      1559
   macro avg       0.52      0.54      0.53      1559
weighted avg       0.74      0.78      0.75      1559



/Users/siglesias/.pyenv/versions/3.8.0/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/siglesias/.pyenv/versions/3.8.0/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/siglesias/.pyenv/versions/3.8.0/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_sta

In [23]:
print(test_df['disease'].value_counts())

0    885
1    594
2     80
Name: disease, dtype: int64
